# Tugas 2: POST Tagging
Kode untuk melakukan post tagging menggunakan metode: Baseline, Viterbi dan Klasifikasi.

In [1]:
# Import library yang diperlukan
import numpy
import io
from itertools import permutations
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline


from sklearn.naive_bayes import MultinomialNB

read training file and create 

In [2]:
# Membaca file dan proses init table
def read_file_init_table(fname):
    tag_count = {}
    tag_count['<start>'] = 0
    word_tag = {}
    tag_trans = {}
    with open(fname) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    idx_line = 0
    is_first_word = 0
    
    while idx_line < len(content):
        prev_tag = '<start>'
        
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                if content_part[1] in tag_count:
                    tag_count[content_part[1]] += 1
                else:
                    tag_count[content_part[1]] = 1
                    
                current_word_tag = content_part[0].lower()+'|'+content_part[1]
                if current_word_tag in word_tag:
                    word_tag[current_word_tag] += 1
                else:    
                    word_tag[current_word_tag] = 1
                    
                if is_first_word == 1:
                    current_tag_trans = '<start>,'+content_part[1]
                    is_first_word = 0
                else:
                    current_tag_trans = prev_tag+','+content_part[1]
                    
                if current_tag_trans in tag_trans:
                    tag_trans[current_tag_trans] += 1
                else:
                    tag_trans[current_tag_trans] = 1                    
                prev_tag = content_part[1]   
                
            else:
                tag_count['<start>'] += 1
                is_first_word = 1
            idx_line = idx_line + 1

        idx_line = idx_line+1       
    return tag_count, word_tag, tag_trans

In [3]:
# Memanggil fungsi init table yang dibangun dari dataset. Init table menghasilkan: tag count, word tag dan jumlahnya dan tag transition dari satu tag ke tag lain

tag_count, word_tag, tag_trans = read_file_init_table('Indonesian_Manually_Tagged_Corpus_ID.tsv')

In [4]:
# Fungsi untuk membangun tabel transition yang berisikan probabilitynya
def create_trans_prob_table(tag_trans, tag_count):
    trans_prob = {}
    for tag1 in tag_count.keys():
        for tag2 in tag_count.keys():
            trans_idx = tag1+','+tag2
            if trans_idx in tag_trans:
                trans_prob[trans_idx] = tag_trans[trans_idx]/tag_count[tag1]
    return trans_prob

In [5]:
trans_prob = create_trans_prob_table(tag_trans, tag_count)

In [6]:
# Fungsi untuk membangun tabel emission
def create_emission_prob_table(word_tag, tag_count):
    emission_prob = {}
    for word_tag_entry in word_tag.keys():
        word_tag_split = word_tag_entry.split('|')
        current_word = word_tag_split[0]
        current_tag = word_tag_split[1]
        emission_key = current_word+','+current_tag
        emission_prob[emission_key] = word_tag[word_tag_entry]/tag_count[current_tag]    
    return emission_prob

In [7]:
emission_prob = create_emission_prob_table(word_tag, tag_count)

In [8]:
# Fungsi untuk mencari tag dari suatu kata pada kalimat menggunakan viterbi.

def viterbi(trans_prob, emission_prob, tag_count, sentence):
    #initialization
    viterbi_mat = {}
    tag_sequence = []
    
    max_before = {}
    for tag in tag_count:
        max_before[tag] = 1
    
    sentence_words = sentence#.split()
    for i in range(0,len(sentence_words)):
        # print(sentence_words[i])
        word_check = sentence_words[i]
        for tag in tag_count:
            max_curr = 0
            for tag2 in tag_count:
                # Kalkulasi viterbi, nilai probability dari kata sebelumnya * transisi * emisi
                cal = max_before.get(tag2, 1) * trans_prob.get( tag + ','+  tag2, 1) * emission_prob.get(sentence_words[i] + ',' + str(tag), 1)
                if (max_curr < cal):
                    max_curr = cal
            max_before[tag] = max_curr
            
            
        max_output = max_before.copy()
        max_value = {}
        max_value['max'] = 0
        max_value['tag'] = ''
        max_value['word'] = word_check
        # mencari nilai maksimum probability suatu kata pada setiap kata pada kalimat.
        for j in max_output:
            if ((max_output[j] < 1) and (max_output[j] > max_value['max'])  ):
                max_value['max'] = max_output[j]
                max_value['tag'] = j
        # probability tertinggi merupakan tag dari kata tersebut
        tag_sequence.append(max_value)
        
    # Mengembalikan tag dari setiap kata dalam bentuk list
    return tag_sequence

In [9]:
# Membaca data test
def read_datatest_viterbi(fname):
    sentences = []
    tags = []
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    idx_line = 0
    while idx_line < len(content):
        sent = []
        tag = []
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                sent.append(content_part[0])
                tag.append(content_part[1])
            idx_line = idx_line + 1
        sentences.append(sent)
        tags.append(tag)
        idx_line = idx_line+2        
    return sentences, tags

In [10]:
# persiapkan y_test yang merupakan tag asli dari kata
y_test = []
i = 0
for tagss in tags:
    for tag in tagss:
        y_test.append(tag)
        i+= 1
        
print(i)

NameError: name 'tags' is not defined

In [ ]:
# persiapkan y_predict yang merupakan tag prediksi dari model viterbi suatu kata
i = 0
y_predict = []
for tags in tags_persentence:
    for tag in tags:
        y_predict.append(tag)
        i+= 1
        
print(i)

In [ ]:
correct = 0
for i in range(len(y_test)):
    # jika tag suatu kata tidak dapat diprediksi oleh model viterbi 
    # karena kata tersebut tidak terkandung pada data latih, 
    # maka gunakan model baseline untuk mencari tag kata tersebut
    # jika model baseline tidak dapat manangani kata tersebut, secara otomatis di set menjadi NN
    if (y_predict[i]['tag'] == ''):
        try:
            y_predict[i]['tag']  = findTag(y_predict[i]['word'])
        except:
            y_predict[i]['tag']  = 'NN'
        
    if y_test[i] == y_predict[i]['tag']:
        correct += 1
print('Akurasi viterbi: ',correct / 499 * 100)

# print(y_predict)

## Model viterbi menghasilkan akurasi 86.97% 
akurasi lebih baik dari model baseline, karena viterbi memperhatikan probability kata sebelumnya

# BASELINE

In [ ]:
# Fungsi untuk mencari semua tag yang ada
def getTagThatPossible(word):
    collection = []
    for item in emission_prob:
        if (item.split(',')[0] == word):
            collection.append(item.split(',')[1])
    return collection

In [ ]:
# Model baseline dengan mencari kemunculan tag terbanyak suatu kata
def findTag(word):
    possibleTag = getTagThatPossible(word)
    maxProb = 0
    maxTag = ''
    for item in possibleTag:
        key = word + ',' + item
        if (emission_prob.get(key, 0) > maxProb):
            maxTag = item 
    return maxTag

In [ ]:
sentences, tags = read_datatest_viterbi('dataset_test.tsv')
tags_predict = []

# mulai penghitungan baseline

for sentence in range(len(sentences)):
    for word in range(len(sentences[sentence])):
        word =  sentences[sentence][word].lower()
        tags_predict.append(findTag(word))        

y_predict = tags_predict

In [ ]:
# Persiapkan data test untuk menghitung akurasi
y_test = []
i = 0
for tagss in tags:
    for tag in tagss:
        y_test.append(tag)
        i+= 1
        
print(i)

In [ ]:
# Hitung akurasi dari model baseline
correct = 0
for i in range(len(y_test)):
    if (y_predict[i] == ''):
        y_predict[i]  = 'NN'
        
    if y_test[i] == y_predict[i]:
        correct += 1
print('Akurasi Baseline: ',correct / 499 * 100)

### Akurasi baseline: 67.73 %, dibawah dari model viterbi dan klasifikasi

## KLASIFIKASI

In [ ]:
# baca dataset
def read_dataset(fname):
    sentences = []
    tags = []
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    idx_line = 0
    while idx_line < len(content):
        sent = []
        tag = []
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                sent.append(content_part[0])
                tag.append(content_part[1])
            idx_line = idx_line + 1
        sentences.append(sent)
        tags.append(tag)
        idx_line = idx_line+2        
    return sentences, tags

# ekstraksi fitur yang digunakan, fitur- fitur yang digunakan adalah:
# kata tersebut secara utuh
# prefix ke 1
# prefix ke 1&2
# prefix ke 1-3
# suffix ke 1
# suffix ke 1&2
# suffix ke 1-3
# kata sebelumnya
# kata setelahnya
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
    }


 
def transform_to_dataset(sentences, tags):
    X, y = [], []
 
    for sentence_idx in range(len(sentences)):
        for index in range(len(sentences[sentence_idx])):
            X.append(features(sentences[sentence_idx], index))
            y.append(tags[sentence_idx][index])
 
    return X, y

In [ ]:
# load data latih dan data train
sentences,tags = read_dataset('dataset_train.tsv')
sentences_test,tags_test = read_dataset('dataset_test.tsv') 

In [ ]:
# ubah data latih ke dalam bentuk dataset yang dapat digunakan
X, y = transform_to_dataset(sentences, tags)
X_test, y_test = transform_to_dataset(sentences_test, tags_test)

In [ ]:
# Klasifier yang digunakan adalah Multinomial Naive Bayes
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier',   MultinomialNB())
])

# latih classifier dengan data train
clf.fit(X, y) 

In [ ]:
# hitung score dari data model kepada data test
print('Akurasi Naive Bayes: ', clf.score(X_test, y_test) * 100)


## Akurasi dari model klasifikasi adalah 94.98
Akurasi tertinggi jika dibanginkan dengan model baseline dan viterbi, hal tersebut dimungkinkan karena pada baseline tidak memperhatikan kata sebelum/ setelah/ imbuhannya sedangkan pada viterbi memperhatikan kata sebelumnya dapat dilihat dari penghitungannya melibatkan nilai dari kata sebelumnya. Pada model klasifikasi menggunakan fitur kata sebelum, kata setelah dan prefix & suffix / imbuhan, sehingga model klasifikasi dapat menghasilkan akurasi yang lebih baik jika dibanginkan dengan model lainnya.